## Visualizing the Price and Load Data via Jupyter Dash

## Fetch the price, load, and price volatility feature data

In [1]:
from jupyter_dash import JupyterDash

import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import pandas as pd
import requests
import cudf

from datetime import datetime
from flask import request
from IPython.display import display, HTML

from cuml import PCA


In [2]:
# get the data
price_path='Jan2021-July2021-price.csv'
load_path='Jan2021-July2021-load.csv'
volatility_path='https://raw.githubusercontent.com/jyu-theartofml/galvanize_datathon/main/processed_data/price_volatility_jan_july.csv'


price_df=cudf.read_csv(price_path, parse_dates=['Time Stamp'])
price_df.index = pd.to_datetime(price_df['Time Stamp'].to_pandas())

load_df=cudf.read_csv(load_path, parse_dates=['Time Stamp'])
load_df.index = pd.to_datetime(load_df['Time Stamp'].to_pandas())

volatility_df=cudf.read_csv(volatility_path)


In [3]:
print(price_df.shape)
price_df.head()

(907995, 10)


,Time Stamp,Name,PTID,LBMP ($/MWHr),Marginal Cost Losses ($/MWHr),Marginal Cost Congestion ($/MWHr),weekday,hour,month,dayofweek
Time Stamp,,,,,,,,,,
2021-01-01 00:05:00,2021-01-01 00:05:00,CAPITL,61757,28.83,2.06,0.0,4,0,1,Friday
2021-01-01 00:05:00,2021-01-01 00:05:00,CENTRL,61754,27.11,0.35,0.0,4,0,1,Friday
2021-01-01 00:05:00,2021-01-01 00:05:00,DUNWOD,61760,29.02,2.25,0.0,4,0,1,Friday
2021-01-01 00:05:00,2021-01-01 00:05:00,GENESE,61753,26.29,-0.48,0.0,4,0,1,Friday
2021-01-01 00:05:00,2021-01-01 00:05:00,H Q,61844,25.99,-0.78,0.0,4,0,1,Friday


In [4]:
print(load_df.shape)
load_df.head()

(653532, 9)


,Time Stamp,Time Zone,Name,PTID,Load,weekday,hour,month,dayofweek
Time Stamp,,,,,,,,,
2021-01-01,2021-01-01,EST,CAPITL,61757,1253.6528,4,0,1,Friday
2021-01-01,2021-01-01,EST,CENTRL,61754,1646.5823,4,0,1,Friday
2021-01-01,2021-01-01,EST,DUNWOD,61760,602.6414,4,0,1,Friday
2021-01-01,2021-01-01,EST,GENESE,61753,1009.6906,4,0,1,Friday
2021-01-01,2021-01-01,EST,HUD VL,61758,1031.5640,4,0,1,Friday


In [5]:
volatility_df.head()

,index,length,mean,var,entropy,lumpiness,stability,flat_spots,hurst,std1st_der,...,firstmin_ac,firstzero_ac,holt_alpha,holt_beta,hw_alpha,hw_beta,hw_gamma,month_int,zone,month
0,0,9094,41.737925,2978.157962,0.774149,5.516761e+08,1281.506579,6,0.046770,26.242856,...,5,928,0.783177,0.021734,<NA>,<NA>,<NA>,1,CAPITL,2021-01-01
1,1,8236,62.901325,3792.912457,0.822366,4.522671e+08,1277.252493,5,0.135222,26.897745,...,9,889,0.709106,0.025666,<NA>,<NA>,<NA>,2,CAPITL,2021-02-01
2,2,9100,36.553865,1895.850091,0.786877,3.799430e+07,836.228066,10,0.080954,20.074640,...,9,1245,0.735714,0.000100,<NA>,<NA>,<NA>,3,CAPITL,2021-03-01
3,3,8827,31.126419,947.322989,0.870041,1.776239e+07,297.824935,12,0.056814,17.241505,...,2,173,0.405734,0.000113,<NA>,<NA>,<NA>,4,CAPITL,2021-04-01
4,4,9157,28.294115,380.042617,0.878728,1.776156e+06,115.092036,11,0.041948,11.079597,...,7,383,0.193572,0.000101,<NA>,<NA>,<NA>,5,CAPITL,2021-05-01


In [6]:
merged_df=price_df.merge(load_df[['Load', 'Time Stamp', 'Name']], on=['Time Stamp', 'Name'], how='left')
merged_df['datetime']=cudf.Series(merged_df['Time Stamp'].to_pandas().dt.date)
                                

In [7]:
merged_df.head()

,Time Stamp,Name,PTID,LBMP ($/MWHr),Marginal Cost Losses ($/MWHr),Marginal Cost Congestion ($/MWHr),weekday,hour,month,dayofweek,Load,datetime
0,2021-01-01 09:30:00,GENESE,61753,21.17,-0.50,0.00,4,9,1,Friday,1016.0542,2021-01-01
1,2021-01-01 09:35:00,HUD VL,61758,23.40,1.75,0.00,4,9,1,Friday,1000.7526,2021-01-01
2,2021-01-01 09:30:00,HUD VL,61758,23.42,1.75,0.00,4,9,1,Friday,1019.4787,2021-01-01
3,2021-01-01 09:35:00,LONGIL,61762,36.47,2.21,-12.61,4,9,1,Friday,1862.3330,2021-01-01
4,2021-01-01 09:35:00,WEST,61752,20.89,-0.76,0.00,4,9,1,Friday,1534.0290,2021-01-01


In [8]:
load_summary=merged_df.groupby(['datetime','Name']).agg({'Load': 'sum'}).reset_index()
load_summary.head()

,datetime,Name,Load
0,2021-03-24,NORTH,1.719812e+05
1,2021-01-11,NORTH,1.949250e+05
2,2021-07-21,N.Y.C.,2.074864e+06
3,2021-07-16,GENESE,3.828626e+05
4,2021-02-14,NORTH,2.024162e+05


####  Get heatmap for daily load consumption

In [12]:
layout = {'title': {'text':'Daily load consumption'},
         'template': 'plotly_dark',
         'paper_bgcolor': '#404040',
         'plot_bgcolor': '#404040',
         'font_color': 'white'}
fig=go.Figure(data=go.Heatmap(
    z=load_summary['Load'].to_array(),
    x=load_summary['datetime'].to_array(),
    y=load_summary['Name'].to_array(),
    colorscale='agsunset'),  layout=layout)

## alternatively use `update_layout` for title, paper_bgcolor, etc


#### Get PCA components from the extracted features in volatility_df

In [13]:
ls_features = ['lumpiness', 'entropy', 'seasonality_strength', 'stability', 'level_shift_size', 'mean', 'var']
volatility_sub=volatility_df[ls_features]

In [14]:
pca_float = PCA(n_components = 2)
volatility_transformed=pca_float.fit_transform(volatility_sub)

In [15]:
volatility_transformed['zone']=volatility_df['zone']
volatility_transformed['month']=volatility_df['month']


In [16]:
volatility_transformed.tail()

,0,1,zone,month
100,-2.610816e+08,-614.519690,WEST,2021-03-01
101,-1.016970e+08,-659.840638,WEST,2021-04-01
102,-2.671299e+08,-1173.053406,WEST,2021-05-01
103,-2.194877e+08,476.543632,WEST,2021-06-01
104,-2.661586e+08,-107.274381,WEST,2021-07-01


## Put the data in a Dash plot!
*  show the price time series
*  get the total Load per month for each zone
*  show the stability metric for each month for each zone
*  plot the heatmap of the daily load consumed for each zone

#### configure the app

In [17]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

zone_palette= ["#FF5733", "#6BAED6", "#D8DDF3", "#ABEBC6", "#F8C471" , "#E6E6E6" ,
               "#B7950B", "#B3CCC8", "#CA9BB0", "#F9C8B9", "#F2E1C1", "#DCEFEB",
               "#BB8FCE", "#D8C292", "#FAE5d3 "]

proxy_port = 8050

app = JupyterDash(__name__,
                  external_stylesheets=external_stylesheets,
                  requests_pathname_prefix='/proxy/' + str(proxy_port) + '/')
app.config.suppress_callback_exceptions = True



In [18]:
def get_options(list_zones):
    dict_list = []
    for i in list_zones:
        dict_list.append({'label': i, 'value': i})

    return dict_list


month_ls=[]
for x in merged_df['month'].unique().to_pandas():
    month_ls.append(datetime(year=2021, month=x, day=2))
         

#### Define the layout here

In [19]:

app.layout = html.Div(
    children=[
        html.Div(className='row',
                 children=[
                    html.Div(className='four columns div-user-controls',
                             children=[
                                 html.H3('Energy Operation Data (New York ISO)', 
                                         style={'color': 'white','fontSize': 24,
                                               'marginLeft': 10, 'marginRight': 10}, 
                                        ),
                                 html.P('Pick one or more zones from the dropdown below.',
                                        style={'color': 'white',
                                              'marginLeft': 10, 'marginRight': 10}),
                                 html.Div(
                                     className='div-for-dropdown',
                                     children=[
                                         dcc.Dropdown(id='zoneselector', 
                                                      options=get_options(merged_df['Name'].unique().to_pandas()),
                                                      multi=True, 
                                                      value=[merged_df['Name'].sort_values().to_pandas()[0]],
                                                      style={'backgroundColor': '#404040'},
                                                      className='zoneselector'
                                                      ),
                                     ],
                                     style={'color': '#404040'})
                                ],
                             style={'backgroundColor': '#404040', 'width': '30%' }
                             ),
                     
                    html.Div(className='eight columns div-for-charts bg-grey',
                             children=[
                                 dcc.Graph(id='timeseries', config={'displayModeBar': False}, 
                                           animate=True)
                             ], style={'backgroundColor': '#404040'}),
                
                              ]
                ),
        # new div for the stability and PCA
        html.Div(className='row',
            children=[
                  html.Div(className='four columns div-user-controls',
                             children=[                                 
                                 
                                 html.Div(
                                     className='div-for-pca',
                                     children=[
                                         dcc.Graph(id='pcaplot', config={'displayModeBar': False}, animate=True)
                                     ], 
                                     style={'backgroundColor': '#404040'})
                                 ],
                                   
                             style={'backgroundColor': '#404040', 'width': '30%' }
                             ),
                
                
                html.Div(className='eight columns div-for-charts bg-grey',
                             children=[
                                 dcc.Graph(id='stabilityplot', config={'displayModeBar': False}, 
                                           animate=True)
                             ], style={'backgroundColor': '#404040'}),
            ]
                ),
                    
        # new div for the barchart
        html.Div(className='row',
            children=[
                 html.H3(),
                html.Div(className='row', 
                        children=[ html.Div(className='row',
                             children=[
                                 dcc.Graph(id='barchart', config={'displayModeBar': False}, animate=True)
                             ], style={'backgroundColor': '#404040'})
                                 ],),
                
               
                
        ], ),
        
    # new div for the heatmap
         html.Div(className='row',
            children=[
                 html.H3(),
                html.Div(className='row', 
                        children=[ html.Div(className='row',
                             children=[
                                 dcc.Graph(id='heatmap', config={'displayModeBar': False}, animate=True, 
                                       figure=fig    )
                             ], style={'backgroundColor': '#404040'}, )
                                 ],),
                
               
                
        ], ),
    ])


# Callback for timeseries price
@app.callback(Output('timeseries', 'figure'),
              [Input('zoneselector', 'value')])
def update_graph(selected_dropdown_value):
    trace1 = []
    df_sub = merged_df
     
    for zone in selected_dropdown_value:
      
        trace1.append(go.Scattergl(x=df_sub[df_sub['Name'] == zone].index.to_array(),
                                 y=df_sub[df_sub['Name'] == zone]['LBMP ($/MWHr)'].to_array(),
                                 mode='lines',
                                 opacity=0.7,
                                 text=df_sub['LBMP ($/MWHr)'].to_array(),
                                 name=zone,
                                 textposition='bottom center'))
        
    traces = [trace1]
    data = [val for sublist in traces for val in sublist]
    figure = {'data': data,
              'layout': go.Layout(
                  colorway=zone_palette,
                  template='plotly_dark',
                  paper_bgcolor='rgba(0, 0, 0, 0)',
                  plot_bgcolor='rgba(0, 0, 0, 0)',
                  margin={'b': 15},
                  hovermode='x',
                  autosize=True,
                  title={'text': 'Price LBMP($/MWHr), Jan-July 2021', 'font': {'color': 'white'}, 'x': 0.5},
                  xaxis={'range': [df_sub.index.min(), df_sub.index.max()]},
              ),

              }

    return figure

# Callback for timeseries price
@app.callback(Output('pcaplot', 'figure'),
              [Input('zoneselector', 'value')])
def update_pca(selected_dropdown_value):
    trace1 = []
     
    for zone in selected_dropdown_value:
      
        trace1.append(go.Scattergl(x=volatility_transformed[volatility_transformed['zone'] == zone][0].to_array(),
                                 y=volatility_transformed[volatility_transformed['zone'] == zone][1].to_array(),
                                 mode='markers',
                                 opacity=0.7,
                                 name=zone,
                                 text=volatility_transformed['month'].to_array(),
                                 textposition='bottom center'))
        
    traces = [trace1]
    data = [val for sublist in traces for val in sublist]
    figure = {'data': data,
              'layout': go.Layout(
                  colorway=zone_palette,
                  template='plotly_dark',
                  paper_bgcolor='rgba(0, 0, 0, 0)',
                  plot_bgcolor='rgba(0, 0, 0, 0)',
                  margin={'b': 15},
                  autosize=True,
                  title={'text': 'PCA cluster of the price time series', 'font': {'color': 'white'}, 'x': 0.5 }
              ),

              }

    return figure


@app.callback(Output('barchart', 'figure'),
              [Input('zoneselector', 'value')])
def update_barchart(selected_dropdown_value):

    trace1 = []
    df_sub = merged_df
     
    for zone in selected_dropdown_value:
        zone_df=df_sub[df_sub['Name'] == zone]
        

        load_sum=zone_df.groupby('month').agg({'Load': 'sum'}).reset_index().sort_values(by=['month'])
      
        trace1.append(go.Bar(name=zone, x=month_ls, y=load_sum['Load'].to_array()))
        
        traces = [trace1]
    data = [val for sublist in traces for val in sublist]
    figure = {'data': data,
              'layout': go.Layout(
                  colorway=zone_palette,
                  template='plotly_dark',
                  paper_bgcolor='rgba(0, 0, 0, 0)',
                  plot_bgcolor='rgba(0, 0, 0, 0)',
                  margin={'b': 10},
                  hovermode='x',
                  autosize=True,
                  barmode= 'group',
                  title={'text': 'Total load consumed (monthly)', 'font': {'color': 'white'}, 'x': 0.5},
                  xaxis={'range': [min(month_ls)-pd.DateOffset(months=1), max(month_ls)+pd.DateOffset(days=20)]},
              )

              }

    return figure



# Callback for stabilityplot price
@app.callback(Output('stabilityplot', 'figure'),
              [Input('zoneselector', 'value')])
def update_stability(selected_dropdown_value):
    trace1 = []
    df_sub = volatility_df
     
    for zone in selected_dropdown_value:
      
        trace1.append(go.Scattergl(x=df_sub[df_sub['zone'] == zone]['month'].to_array(),
                                 y=df_sub[df_sub['zone'] == zone]['entropy'].to_array(),
                                 mode='lines',
                                 opacity=0.7,
                                 name=zone,
                                 textposition='bottom center'))
        
    traces = [trace1]
    data = [val for sublist in traces for val in sublist]
    figure = {'data': data,
              'layout': go.Layout(
                  colorway=zone_palette,
                  template='plotly_dark',
                  paper_bgcolor='rgba(0, 0, 0, 0)',
                  plot_bgcolor='rgba(0, 0, 0, 0)',
                  margin={'b': 15},
                  hovermode='x',
                  autosize=True,
                  title={'text': 'Entropy/Uncertainy measure of price time series', 'font': {'color': 'white'}, 'x': 0.5},
                  xaxis={'range': [df_sub['month'].min(), df_sub['month'].max()]},
              ),

              }

    return figure





In [20]:
js = "<b style='color: red'>Please click on <a href='/proxy/" + str(proxy_port) + "/' target='_blank'>here</a> to open the dash</b>"
display(HTML(js))
app.run_server(debug=True, use_reloader=False, port=proxy_port)


Dash app running on http://127.0.0.1:8050/proxy/8050/
